In [5]:
import sys
import os

# Make sure your project root is on PYTHONPATH
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)


from models.resnet50_backbone import ResNet50Backbone
from models.vgg19bn_backbone import VGG19BNBackbone

from src.data_loader import ShanghaiTechDataset
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt



## RESNET 

In [9]:
# ====== Part A evaluation (ResNet-50 density-map backbone only) ======

# Ensure project root on PYTHONPATH
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Evaluating on device: {device}")

# Data transform and loader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

val_dataset_A = ShanghaiTechDataset(
    root="../data/ShanghaiTech",
    part="part_A",
    split="test_data",
    transform=transform,
    return_count=False
)
val_loader_A = DataLoader(val_dataset_A, batch_size=1, shuffle=False)

# Import only ResNet50 density backbone
# Instantiate and load weights
model = ResNet50Backbone(pretrained=True).to(device)
weight_path = "../models/pth/part_A_resnet50.pth"
model.load_state_dict(torch.load(weight_path, map_location=device))
model.eval()

# Evaluate
total_mae = 0.0
total_mse = 0.0

with torch.no_grad():
    for img, gt_density in val_loader_A:
        img        = img.to(device)       # B×3×H×W
        gt_density = gt_density.to(device)  # B×1×H×W

        # True count
        gt_count = gt_density.sum(dim=(1,2,3))  # B

        # Predicted density and count
        pred_density = model(img)               # B×1×H×W
        pred_count   = pred_density.sum(dim=(1,2,3))

        # Accumulate
        total_mae += torch.abs(pred_count - gt_count).sum().item()
        total_mse += ((pred_count - gt_count)**2).sum().item()

N    = len(val_dataset_A)
mae  = total_mae / N
rmse = (total_mse / N)**0.5

print(f"Part A — resnet34: MAE = {mae:.3f}, RMSE = {rmse:.3f}")


Evaluating on device: mps
Part A — resnet50: MAE = 128.223, RMSE = 214.273


In [ ]:
# ====== Part A evaluation (VGG19 density-map backbone only) ======

# Ensure project root on PYTHONPATH
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Evaluating on device: {device}")

# Data transform and loader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

val_dataset_A = ShanghaiTechDataset(
    root="../data/ShanghaiTech",
    part="part_A",
    split="test_data",
    transform=transform,
    return_count=False
)
val_loader_A = DataLoader(val_dataset_A, batch_size=1, shuffle=False)

# Import only VGG19 density backbone
# Instantiate and load weights
model = VGG19BNBackbone(pretrained=True).to(device)
weight_path = "../models/pth/part_A_vgg19.pth"
model.load_state_dict(torch.load(weight_path, map_location=device))
model.eval()

# Evaluate
total_mae = 0.0
total_mse = 0.0

with torch.no_grad():
    for img, gt_density in val_loader_A:
        img        = img.to(device)       # B×3×H×W
        gt_density = gt_density.to(device)  # B×1×H×W

        # True count
        gt_count = gt_density.sum(dim=(1,2,3))  # B

        # Predicted density and count
        pred_density = model(img)               # B×1×H×W
        pred_count   = pred_density.sum(dim=(1,2,3))

        # Accumulate
        total_mae += torch.abs(pred_count - gt_count).sum().item()
        total_mse += ((pred_count - gt_count)**2).sum().item()

N    = len(val_dataset_A)
mae  = total_mae / N
rmse = (total_mse / N)**0.5

print(f"Part A — VGG19bn: MAE = {mae:.3f}, RMSE = {rmse:.3f}")


Evaluating on device: cpu
Part A — resnet50: MAE = 136.715, RMSE = 213.853
